# Lesson 10 supplement - More on pandas - merge() - concat() - rank() -drop()

#### Start always by importing the libraries that will be used

In [52]:
import pandas as pd
import numpy as np

In [ ]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

#set Google Drive path
path_gdrive = '/content/gdrive/MyDrive/ENV716_F22_Solutions/A04/'
os.chdir(path_gdrive)

## I. Merge - ON argument

### The pd.merge function can take multiple optional parameters.
#### One important parameter specifies ON which column do we want to merge the two data frames, or if we want to merge it on the rows.
#### In the case of the GEN and PLNT dataframes discussed in L9, we did not need to specify the ON parameter, because the two dataframes had only one column in common which was the plant code, and this column had the same name in both data frames. So merge() identified that common column and merged the two datasets ON that column. But in the example below we will see an illustration of the use of the parameter on.

In [53]:
# Rather than reading an excel spreadsheet as a dataframe I am defining a dictionary and then converting it into a dataframe
# The dictionary called data1 has three pairs of keys and values. Each pair will become a column of the dataframe
data1= {
  "PLANTname": ["Plant1", "Plant2", "Plant3", "Plant4", "Plant5","Plant6", "Plant7", "Plant8", "Plant9", "Plant10" ],
  "PLANTage": [17, 44, 22, 26, 30, 25, 24, 34, 40, 45],
  "PLANTfuel":["Coal","NG","Sun","Nuclear","Wind","Coal","NG","Sun","Nuclear","Wind"], 
  "PLANTregion": ["North", "South", "East", "West", "North", "South", "East", "West", "East", "West"]}
#Here another dictionary:
data2={  
    "PLANTname": ["Plant1", "Plant2", "Plant3", "Plant4", "Plant5","Plant6", "Plant7", "Plant8", "Plant9", "Plant10" ],
    "PLANTCF": [75, 65, 30, 95, 41, 73, 67, 31, 92, 40,],
    "PLANTfuel":["Coal","NG","Sun","Nuclear","Wind","Coal","NG","Sun","Nuclear","Wind"], 
    "FuelReliability":["Very High","High","Medium","Very High","Low","Very High","High","Medium","Very High","Low"]}
#Now I convert the dictionaries to data frames
df1=pd.DataFrame(data1)
df2=pd.DataFrame(data2)

In [54]:
df1

,PLANTname,PLANTage,PLANTfuel,PLANTregion
0,Plant1,17,Coal,North
1,Plant2,44,NG,South
2,Plant3,22,Sun,East
3,Plant4,26,Nuclear,West
4,Plant5,30,Wind,North
5,Plant6,25,Coal,South
6,Plant7,24,NG,East
7,Plant8,34,Sun,West
8,Plant9,40,Nuclear,East
9,Plant10,45,Wind,West


In [55]:
df2

,PLANTname,PLANTCF,PLANTfuel,FuelReliability
0,Plant1,75,Coal,Very High
1,Plant2,65,NG,High
2,Plant3,30,Sun,Medium
3,Plant4,95,Nuclear,Very High
4,Plant5,41,Wind,Low
5,Plant6,73,Coal,Very High
6,Plant7,67,NG,High
7,Plant8,31,Sun,Medium
8,Plant9,92,Nuclear,Very High
9,Plant10,40,Wind,Low


#### If we want to merge the two dataframes, we can use merge(df1,df2) without specifying any other parameters because there is at least one column that has the same name in both dataframes. In this case, the columns PLANTname and PLANTfuel have the same name and the exact same values in each row. For this reason, it does not matter if we do not specify ON what column we want to merge them. However, if they did not have the same values in the PLANTfuel column, for example, and we do not specify if we want to merge on PLANTname or on PLANTfuel, we could get unexpected results. If we want to keep the info on the fuel reported for each plant in both dataframes it is better to secify that we want to merge on the PLANTname.  Now, since the same plants are in both dataframes, how we merge them does not matter. Sometimes, HOW to merge them matters. See the examples below to learn (but understanding of that concept is not needed for A4)

In [56]:
df_mergedOneTwo=pd.merge(df1,df2)
df_mergedOneTwo

,PLANTname,PLANTage,PLANTfuel,PLANTregion,PLANTCF,FuelReliability
0,Plant1,17,Coal,North,75,Very High
1,Plant2,44,NG,South,65,High
2,Plant3,22,Sun,East,30,Medium
3,Plant4,26,Nuclear,West,95,Very High
4,Plant5,30,Wind,North,41,Low
5,Plant6,25,Coal,South,73,Very High
6,Plant7,24,NG,East,67,High
7,Plant8,34,Sun,West,31,Medium
8,Plant9,40,Nuclear,East,92,Very High
9,Plant10,45,Wind,West,40,Low


#### Now, lets imagine that before merging the two data frames, for some reason, we summarized each one in a pivot table. We wanted a summary of power generation by fuel types from df1. We created a pivot table with df1:

In [57]:
pivot1=pd.pivot_table(df1,index="PLANTfuel",values="PLANTage",columns="PLANTregion",aggfunc=np.mean,fill_value="")
pivot1

PLANTregion,East,North,South,West
PLANTfuel,,,,
Coal,,17.0,25.0,
NG,24.0,,44.0,
Nuclear,40.0,,,26.0
Sun,22.0,,,34.0
Wind,,30.0,,45.0


#### Note that pivot1 is a dataframe. In this dataframe there are 4 columns: East, North, South, West
#### PLANTfuel is no longer a column. It is now and INDEX. We can see a list of the column names with the command: columns=pivot1.columns.values.tolist()

In [58]:
columns=pivot1.columns.values.tolist()
columns

['East', 'North', 'South', 'West']

#### Lets pretend we also wanted to create a pivot table with df2. This summarizes the information for each fuel as well

In [59]:
pivot2=pd.pivot_table(df2,index="PLANTfuel",values="PLANTCF",columns="FuelReliability",aggfunc=np.mean,fill_value="")
pivot2

FuelReliability,High,Low,Medium,Very High
PLANTfuel,,,,
Coal,,,,74.0
NG,66.0,,,
Nuclear,,,,93.5
Sun,,,30.5,
Wind,,40.5,,


##### In the pivot2 dataframe the Fuel is no longer a column but an index

In [60]:
columns=pivot2.columns.values.tolist()
columns

['High', 'Low', 'Medium', 'Very High']

#### If we try to merge pivot1 and pivot 2, just using the default arguments, we will get an ERROR, because there is not a column with the same name in both dataframes. It does not matter that the indexes (i.e., PLANTfuel) in both dataframes are the same

In [45]:
finaldf=pd.merge(pivot1,pivot2)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

### But if we specify that we want to merge the two dataframes ON the fuels (i.e., on PLANTfuel which is an index), the dataframes will be merged correctly

In [46]:
finaldf=pd.merge(pivot1,pivot2,on="PLANTfuel")
finaldf

,East,North,South,West,High,Low,Medium,Very High
PLANTfuel,,,,,,,,
Coal,,17.0,25.0,,,,,74.0
NG,24.0,,44.0,,66.0,,,
Nuclear,40.0,,,26.0,,,,93.5
Sun,22.0,,,34.0,,,30.5,
Wind,,30.0,,45.0,,40.5,,


In [51]:
finaldf["suma regiones"]=finaldf.iloc[:,0:3].sum(axis=1)
finaldf

,East,North,South,West,High,Low,Medium,Very High,suma regiones
PLANTfuel,,,,,,,,,
Coal,,17.0,25.0,,,,,74.0,0.0
NG,24.0,,44.0,,66.0,,,,0.0
Nuclear,40.0,,,26.0,,,,93.5,0.0
Sun,22.0,,,34.0,,,30.5,,0.0
Wind,,30.0,,45.0,,40.5,,,0.0


## II. Concat - as an alternative to the merge used in the example above - index argument
#### In the example above, we wanted to merge two dataframes that had the same index. Here we can use concat. 
#### concat() takes as an argument, a list of dataframes to concatenate. This means, you can concatenate several dataframes, one afer the other. The default is to place each dataframe BELOW the previous dataframe. This is useful when you have the same columns but your rows correspond to different observations.  This means you are concatenating along the INDEX which is AXIS=0.  If instead of placing one data frame below the other we want to put the next dataframe to the RIGHT of the other, we want to concatenate on the columns which is AXIS=1.
#### There are many other optional parameters. The sintaxis is:  pandas.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=True)[source]

In [62]:
df_concatenatedOneTwo=pd.concat([pivot1,pivot2], axis=1)
df_concatenatedOneTwo

,East,North,South,West,High,Low,Medium,Very High
PLANTfuel,,,,,,,,
Coal,,17.0,25.0,,,,,74.0
NG,24.0,,44.0,,66.0,,,
Nuclear,40.0,,,26.0,,,,93.5
Sun,22.0,,,34.0,,,30.5,
Wind,,30.0,,45.0,,40.5,,


#### The result above is the same as when we used merge() specifying on="PLANTfuel"

## III. Merge - How argument

### The pd.merge function can take multiple optional paramters.
#### One important parameter specifies HOW to merge the dataframes. The type of merge can be "left", "right", "outers", or the dafault "inner"
#### In the case of the GEN and PLNT dataframes discussed in L9, the method did not matter, because each plant has at least one generator and each generator belongs to one plant listed in the plant data frame.
#### But in the examples below, how the dfs are merged matters

### 1. Create data frames. 
#### In this case, rather than reading them from Excel, we are going to define some dictionaries. 
#### Each key in the dictionary will later become a column name in the data frame. The values of the dict will be the rows

In [13]:
data1 = {
  "PLANTname": ["Plant1", "Plant2", "Plant3"],
  "PLANTage": [10, 24, 30]
}

data2 = {
  "PLANTname": ["Plant4", "Plant5", "Plant6", "Plant7"],
  "PLANTage": [17, 44, 22, 26]
}
data3= {
  "PLANTname": ["Plant1", "Plant2", "Plant3", "Plant4", "Plant5"],
  "PLANTage": [17, 44, 22, 26, 30],
  "PLANTFuel": ["Coal", "Coal", "NG", "Sun", "Wind"]
}
data4= {
  "PLANTname": ["Plant1", "Plant2", "Plant3", "Plant4", "Plant5"],
  "PLANTage": [10, 24, 30, 26, 30],
  "PLANTFuel": ["Coal", "Coal", "NG", "Sun", "Wind"]
}
data5 = {
  "PLANTname": ["Plant1", "Plant2", "Plant3","Plant4", "Plant5", "Plant6", "Plant7"],
  "PLANTcapacity": [500, 400, 300, 100, 200, 250, 350]
}
data6 =  {
  "PLANTname": ["Plant1", "Plant1", "Plant2","Plant2", "Plant2", "Plant3", "Plant4", "Plant5", "Plant5", "Plant6", "Plant7"],
  "GENCF": [500, 400, 300, 100, 200, 250, 350, 180, 450, 510, 380]
}
type(data1)

dict

### 2. Create the dataframes from the dictionaries

In [14]:
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)
df4 = pd.DataFrame(data4)
df5 = pd.DataFrame(data5)
df6 = pd.DataFrame(data6)

In [15]:
df1

,PLANTname,PLANTage
0,Plant1,10
1,Plant2,24
2,Plant3,30


### 3. Merging df1 and df2. Common columns: PLANTname and PLANTage. Common rows: None. So the intersection is null.
#### An inner merge will produce a dataframe with zero rows

In [16]:
df_InnerOneTwo = pd.merge(df1, df2, how='inner') # same as df1.merge(df2,how='inner')
df_InnerOneTwo.shape

(0, 2)

#### A right merge will produce a data frame with all the rows in df2 which is the right df

In [17]:
df_RightOneTwo=pd.merge(df1, df2, how='right')
df_RightOneTwo

,PLANTname,PLANTage
0,Plant4,17
1,Plant5,44
2,Plant6,22
3,Plant7,26


#### A left merge will produce a data frame with all the rows in df1 which is the left df

In [18]:
df_LeftOneTwo=pd.merge(df1, df2, how='left')
df_LeftOneTwo

,PLANTname,PLANTage
0,Plant1,10
1,Plant2,24
2,Plant3,30


#### A "outer" merge will produce a dataframe with all the rows in each df1 and df2. No data is lost with "outer"

In [19]:
df_OuterOneTwo=pd.merge(df1, df2, how='outer')
df_OuterOneTwo

,PLANTname,PLANTage
0,Plant1,10
1,Plant2,24
2,Plant3,30
3,Plant4,17
4,Plant5,44
5,Plant6,22
6,Plant7,26


#### Another way to perform an outer merge (or join) is using the pd.concat()

In [20]:
df_ConcatOneTwo=pd.concat([df1, df2])
df_ConcatOneTwo

,PLANTname,PLANTage
0,Plant1,10
1,Plant2,24
2,Plant3,30
0,Plant4,17
1,Plant5,44
2,Plant6,22
3,Plant7,26


### 4. Merge InnerOneThree with df5
#### The two dataframes have 1 column in common, PLANTname, and 7 rows in common (plants 1 through). An inner join will keep all the intersection rows, which is all the rows

In [21]:
df_InnerOneThreeFive=pd.merge(df_OuterOneTwo,df5) #remeber that the default is "inner". So omitting the "how" means "inner"
df_InnerOneThreeFive

,PLANTname,PLANTage,PLANTcapacity
0,Plant1,10,500
1,Plant2,24,400
2,Plant3,30,300
3,Plant4,17,100
4,Plant5,44,200
5,Plant6,22,250
6,Plant7,26,350


#### An outer merge would have given the same result

In [22]:
df_OuterOneThreeFive=pd.merge(df_OuterOneTwo,df5) 
df_OuterOneThreeFive

,PLANTname,PLANTage,PLANTcapacity
0,Plant1,10,500
1,Plant2,24,400
2,Plant3,30,300
3,Plant4,17,100
4,Plant5,44,200
5,Plant6,22,250
6,Plant7,26,350


### Concat would also work here, but the default will not give us the desired table
#### Because the default is to concatenate the dataframes placing the rows of the second df after the rows of the first

In [23]:
df_ConcatOneThreeFive=pd.concat([df_OuterOneTwo,df5]) #remember that the first argument of concat is a list of dataframes [,]
df_ConcatOneThreeFive

,PLANTname,PLANTage,PLANTcapacity
0,Plant1,10.0,NaN
1,Plant2,24.0,NaN
2,Plant3,30.0,NaN
3,Plant4,17.0,NaN
4,Plant5,44.0,NaN
5,Plant6,22.0,NaN
6,Plant7,26.0,NaN
0,Plant1,NaN,500.0
1,Plant2,NaN,400.0
2,Plant3,NaN,300.0


#### For concat to work we need to specify the axis. Axis=0 is the default, means to place rows after rows. Axis=1 means to place the columns of the second df, to the right of the columns for the first df

In [24]:
df_ConcatOneThreeFive=pd.concat([df_OuterOneTwo,df5],axis=1) #remember that the first argument of concat is a list of dataframes [,]
df_ConcatOneThreeFive

,PLANTname,PLANTage,PLANTname,PLANTcapacity
0,Plant1,10,Plant1,500
1,Plant2,24,Plant2,400
2,Plant3,30,Plant3,300
3,Plant4,17,Plant4,100
4,Plant5,44,Plant5,200
5,Plant6,22,Plant6,250
6,Plant7,26,Plant7,350


### Experiment with merge and concat with df3, df4, df5

In [25]:
df_InnerOneThree=pd.merge(df1,df3,how='inner')
df_InnerOneThree.shape

(0, 3)

In [26]:
df_LeftOneThree=pd.merge(df1,df3,how='left')
df_LeftOneThree

,PLANTname,PLANTage,PLANTFuel
0,Plant1,10,NaN
1,Plant2,24,NaN
2,Plant3,30,NaN


In [27]:
df_RightOneThree=pd.merge(df1,df3,how='right')
df_RightOneThree

,PLANTname,PLANTage,PLANTFuel
0,Plant1,17,Coal
1,Plant2,44,Coal
2,Plant3,22,NG
3,Plant4,26,Sun
4,Plant5,30,Wind


In [28]:
df_InnerOneFour=pd.merge(df1,df4,how='inner')
df_InnerOneFour.shape
df_InnerOneFour

,PLANTname,PLANTage,PLANTFuel
0,Plant1,10,Coal
1,Plant2,24,Coal
2,Plant3,30,NG


In [29]:
df_LeftOneFour=pd.merge(df1,df4,how='left')
df_LeftOneFour.shape
df_LeftOneFour

,PLANTname,PLANTage,PLANTFuel
0,Plant1,10,Coal
1,Plant2,24,Coal
2,Plant3,30,NG


In [30]:
df_RightOneFour=pd.merge(df1,df4,how='right')
df_RightOneFour.shape
df_RightOneFour

,PLANTname,PLANTage,PLANTFuel
0,Plant1,10,Coal
1,Plant2,24,Coal
2,Plant3,30,NG
3,Plant4,26,Sun
4,Plant5,30,Wind


## IV. Rank()

#### Taking the rank is the same as taking other statistics of a dataframe's numerical column. Lets create a new column to one of the dfs created before.  rank() has several arguments. Explore them 

In [31]:
#Lets take one of the dfs created above
df_OuterOneThreeFive

,PLANTname,PLANTage,PLANTcapacity
0,Plant1,10,500
1,Plant2,24,400
2,Plant3,30,300
3,Plant4,17,100
4,Plant5,44,200
5,Plant6,22,250
6,Plant7,26,350


#### Lets create a new column with the rank by capacity, from largest to lowest

In [32]:
df_OuterOneThreeFive['RankCap']=df_OuterOneThreeFive["PLANTcapacity"].rank(ascending=False)
df_OuterOneThreeFive

,PLANTname,PLANTage,PLANTcapacity,RankCap
0,Plant1,10,500,1.0
1,Plant2,24,400,2.0
2,Plant3,30,300,4.0
3,Plant4,17,100,7.0
4,Plant5,44,200,6.0
5,Plant6,22,250,5.0
6,Plant7,26,350,3.0


## V. drop()

#### We can drop a row, if we are not interested in keeping it. For example,lets drop Plant6, which is in row 5

In [69]:
df_new=df_OuterOneThreeFive.drop(index=5, inplace=False)#If inplace equal true then it modifyes this df instead of creating a copy

#### Wth inplace-False we created a copy of the df_OuterOneThreeFive that does not have a row for plant6. 

In [70]:
df_new # note that row 5 (index 5) is no longer there, but the rows were not renumbered 

,PLANTname,PLANTage,PLANTcapacity,RankCap
0,Plant1,10,500,1.0
1,Plant2,24,400,2.0
2,Plant3,30,300,4.0
3,Plant4,17,100,7.0
4,Plant5,44,200,6.0
6,Plant7,26,350,3.0


#### But the original dataframe still has all the rows complete

In [71]:
df_OuterOneThreeFive

,PLANTname,PLANTage,PLANTcapacity,RankCap
0,Plant1,10,500,1.0
1,Plant2,24,400,2.0
2,Plant3,30,300,4.0
3,Plant4,17,100,7.0
4,Plant5,44,200,6.0
5,Plant6,22,250,5.0
6,Plant7,26,350,3.0
